<a href="https://colab.research.google.com/github/TalissaMoura/sounding_earth_with_vit/blob/main/notebooks/02_advance_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Mount drive and call condacolab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Download code
!git clone 

In [ ]:
## Run this cell to load/download the datasets.

# #Deselect if you need to download the dataset
# print("--- Start Downloading sounding earth dataset files ---")
# !wget -P ./sounding_earth_with_vit/datasets https://zenodo.org/records/3828124/files/ADVANCE_sound.zip
# !wget -P ./sounding_earth_with_vit/datasets https://github.com/khdlr/SoundingEarth/releases/tag/spectrograms
# print("--- Finish Downloading sounding earth dataset files ---")

## Here I'm using the data that is saved on my Drive. This is recommended for more faster results.
print("--- Descompress the Advance files --- ")
!mkdir -p ./test_advance/datasets
!tar -xvf ./drive/MyDrive/datasets/advance_spectrograms.tar.gz -C ./sounding_earth_with_vit/datasets
!unzip ./drive/MyDrive/datasets/ADVANCE_vision.zip -d ./sounding_earth_with_vit/datasets
print("--- Finish descompress Advance files ---")

Streaming output truncated to the last 5000 lines.
  inflating: ./test_advance/datasets/vision/forest/09144.jpg  
  inflating: ./test_advance/datasets/vision/forest/03534.jpg  
  inflating: ./test_advance/datasets/vision/forest/05140.jpg  
  inflating: ./test_advance/datasets/vision/forest/07147.jpg  
  inflating: ./test_advance/datasets/vision/forest/05276.jpg  
  inflating: ./test_advance/datasets/vision/forest/07523.jpg  
  inflating: ./test_advance/datasets/vision/forest/03994.jpg  
  inflating: ./test_advance/datasets/vision/forest/04429.jpg  
  inflating: ./test_advance/datasets/vision/forest/05861.jpg  
  inflating: ./test_advance/datasets/vision/forest/02487.jpg  
  inflating: ./test_advance/datasets/vision/forest/03421.jpg  
  inflating: ./test_advance/datasets/vision/forest/11263.jpg  
  inflating: ./test_advance/datasets/vision/forest/02043.jpg  
  inflating: ./test_advance/datasets/vision/forest/07060.jpg  
  inflating: ./test_advance/datasets/vision/forest/04664.jpg  
  in

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://bucket-metadados-modelos.s3.us-east-1.amazonaws.com/envs/sounding_earth-4-Linux-x86_64.sh?response-content-disposition=inline&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEPD%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXNhLWVhc3QtMSJIMEYCIQCaYTIzk87XIiwn0%2FNz1uidnoY%2FLwfWqw1wFjtIkqM7%2BgIhAOR%2FHI8hobrY6PbgaxHviPMucCRbs2LbWgTJ06Ko5qT%2FKu0CCKn%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQABoMMTEyNDk5MjY5Mzg5IgxN005g%2BQGVjJ%2F6cOIqwQKOi98w1alXNCFfuGAdmOW3cWbtt9CPEUJvUdfpM2Gum5Cu5meG255YWuM85yuPmkpvPRoX15a0AYdZAelBJBMDkXaYFWLrjT4Nqo3%2F8rN0tWhCAbiiMkzZLK%2BXZ5jO9M0YI%2BbYMC8zLr6k584F9dIMgRIhVkBaCJJ6ZGVKiCM7shc2Neuy1IH9t3H%2BbYjvUS3IwCPTyMmq8btbFk9atW5SuaePlntTbuQd%2FiDiN8%2B%2BOUNPDJqdGH6FBzvEA6a452x31vEn42%2FAVxFKm23huJdfiMp6wdbQ8YPAEz2Y6%2FS5ntj0UftmTKOBtyKJ8pD8piWU%2Fu8vTadTyert7vLCK5%2FO8zSIO%2BLtzlz31P2cOGl%2BhiML67it5rCpKguxMV2tvfrnJ6xlOFGmFPwKAyziEdc06faPmy%2BaG3syDIYgOZeo1MswjqPPrQY6sgK2dz2hSTThg0wbZV7tAfjI59FS5J%2FCIGQUiaE6o%2FrhNG9a4NqjsZoTlRsN6VyquFRMPXipjNB9Jy5u1ZXQRqHeBoh89xlLVafwn

In [ ]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


# 2 - Making imports

In [ ]:
!pwd

/content


In [ ]:
%cd /content/sounding_earth_with_vit

/content/test_advance


In [ ]:
# import libs
!conda env update -n base -f ./sounding_earth_with_vit/environment_v4.yml

In [ ]:
import sys
import numpy as np
import pandas as pd
import json
from argparse import ArgumentParser
from pathlib import Path
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from einops import rearrange
import pickle
import ipywidgets as widgets
#import wandb

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

from lib import get_model, get_loss_function, FullModelWrapper
from config import cfg

In [ ]:
# Improve reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# For GPU
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
import os
os.environ['WANDB__EXECUTABLE']='/usr/local/bin/python3.10'

In [ ]:
import wandb
#key = '998ffb924cd565faa75d442dad813b2cd96d1d0a'
#wandb.login()
!wandb login

wandb: Currently logged in as: talissamoura. Use `wandb login --relogin` to force relogin


In [ ]:
LOW  = np.exp(-15 / 10)
HIGH = np.exp(5 / 10)

In [ ]:
# Define run_id and path to models
run_id = widgets.Text(
    value='',
    placeholder='Wandb run id',
    description='run_id:',
    disabled=False
)

model_path_str = widgets.Text(
    value='',
    placeholder='Type the dir',
    description='path_to_models:',
    disabled=False
)

## show widget
display(run_id)
display(model_path_str)

Text(value='', description='run_id:', placeholder='Wandb run id')

Text(value='', description='path_to_models:', placeholder='Type the dir')

In [ ]:
# CLOSE WIDGETS
run_id.close()
model_path_str.close()

# RUN ID NAME
run_id = run_id.value

# MODEL PATH
PATH_TO_MODELS = Path(model_path_str.value)

print("run name:",run_id)
print("model path:",PATH_TO_MODELS)

run name: test_advance_vit_base_32patch_224_ft_in21k_lr_v02_best
model path: logs/run_vit_base_patch32_224_ftnd_in21k-2


# 3 - Call functions

In [ ]:
def evaluate_advance(model, dev=None,seed=42,run=None,log_artifacts=False):
    if dev is None:
        dev = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda")

    img_encoder = model.img_encoder
    snd_encoder = model.snd_encoder
    loss_function = model.loss_function

    data_root = Path(".") / 'datasets'

    images = list((data_root / 'vision').glob('*/*.jpg'))
    images.sort()

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    normalize = transforms.Normalize(mean=mean, std=std)
    resize = transforms.Resize(size=(224,224))
    imgtransform = transforms.Compose([
        transforms.ToTensor(),resize,normalize
    ])
    torch.set_grad_enabled(False)

    data = []
    for img_path in tqdm(images):
        key = img_path.stem
        label = img_path.parent.name


        snd_path = data_root / 'advance_spectrograms' / label / f'{key}.npy'
        img = imgtransform(Image.open(img_path))
        img = img.to(dev)
        img = img.unsqueeze(0)


        snd = np.load(snd_path).astype(np.float32)
        snd = np.clip((np.exp(snd / 10) - LOW) / (HIGH - LOW), 0, 1)
        snd = torch.from_numpy(snd).to(dev)
        snd = snd.unsqueeze(0).unsqueeze(1)
        snd = resize(snd)
        snd = snd.repeat(1,3,1,1)
        snd = normalize(snd)


        z_img = loss_function.distance_transform(img_encoder(img))[0].cpu().numpy()
        z_snd = loss_function.distance_transform(snd_encoder(snd))[0].cpu().numpy()



        data.append(dict(
            key=key,
            label=label,
            z_img=z_img,
            z_snd=z_snd
        ))




    key   = [d['key'] for d in data]
    label = [d['label'] for d in data]
    z_img = np.stack([d['z_img'] for d in data], axis=0)
    z_snd = np.stack([d['z_snd'] for d in data], axis=0)

    print("z_snd after stack:",z_snd.shape)
    z_img = rearrange(z_img,'b w h -> b (w h)')
    z_snd = rearrange(z_snd,'b w h -> b (w h)')
    print("z_snd after rearrange:",z_snd.shape)




    # Numeric Encoding for labels
    idx2lbl = np.unique(label)
    lbl2idx = {l: i for i, l in enumerate(idx2lbl)}
    label = np.array([lbl2idx[l] for l in label])
    class_weights = np.bincount(label) / len(label)

    #save label index

    with open("label_index.txt","a") as file:
      for l,i in lbl2idx.items():
        file.write(f"{l},{i}\n")
      file.close()

    print("--- save lbl2idex file ---")
    lblidx_file = wandb.Artifact(f"lbl_idx_for_run_{run.id}",type="label_data")
    lblidx_file.add_file(f"./label_index.txt")
    run.log_artifact(lblidx_file)
    print("--- logged labelidx file ---")



    results = []

    trnval_test = train_test_split(key, label, z_img, z_snd, stratify=label, test_size=0.2, random_state=seed)
    key_trnval, key_test, lbl_trnval, lbl_test, Z_img_trnval, Z_img_test, Z_snd_trnval, Z_snd_test = trnval_test

    if log_artifacts and run != None:

      print(f"---- save test data for run {run.id} ----")

      test_data_at = wandb.Artifact(f"test_data_for_run_{run.id}", type="test_data")


      if Path("./test_data").exists():
        np.save(f"./test_data/key_test_data.npy",arr=key_test)
        np.save(f"./test_data/lbl_test_data.npy",arr=lbl_test)
        np.save(f"./test_data/z_snd_test.npy",arr=Z_snd_test)
        np.save(f"./test_data/z_img_test.npy",arr=Z_img_test)
      else:
        Path("./test_data").mkdir(parents=True,exist_ok=True)
        np.save(f"./test_data/key_test_data.npy",arr=key_test)
        np.save(f"./test_data/lbl_test_data.npy",arr=lbl_test)
        np.save(f"./test_data/z_snd_test.npy",arr=Z_snd_test)
        np.save(f"./test_data/z_img_test.npy",arr=Z_img_test)

      test_data_at.add_dir("./test_data")

      run.log_artifact(test_data_at)
      print(f"----  Logged test data for {run.id} ----")



    for data_mode in ['image', 'sound', 'mean', 'concat']:
    # for data_mode in ['concat']:
        print(f'Taking {data_mode} as data basis.')
        for i in tqdm(range(5)):
            seed = 42 * i

            trn_val = train_test_split(key_trnval, lbl_trnval, Z_img_trnval, Z_snd_trnval,
                            stratify=lbl_trnval, test_size=0.1/0.8, random_state=seed)
            key_trn, key_val, lbl_trn, lbl_val, Z_img_trn, Z_img_val, Z_snd_trn, Z_snd_val = trn_val


            if log_artifacts and run != None:
              print(f"---- save val data and train data for run {run.id} ----")
              val_data_at = wandb.Artifact(f"val_data_for_run_{run.id}_{data_mode}_split{i+1}", type="val_data")
              train_data_at = wandb.Artifact(f"train_data_for_run_{run.id}_{data_mode}_split{i+1}", type="train_data")
              path_train_dir =  Path(f"./train_data/{data_mode}/split{i+1}")
              path_val_dir =  Path(f"./val_data/{data_mode}/split{i+1}")

              if path_train_dir.exists():
                np.save(f"./train_data/{data_mode}/split{i+1}/key_trn.npy",arr=key_trn)
                np.save(f"./train_data/{data_mode}/split{i+1}/lbl_trn.npy",arr=lbl_trn)
                np.save(f"./train_data/{data_mode}/split{i+1}/z_snd_trn.npy",arr=Z_snd_trn)
                np.save(f"./train_data/{data_mode}/split{i+1}/z_img_trn.npy",arr=Z_img_trn)
              else:
                  path_train_dir.mkdir(parents=True,exist_ok=True)
                  np.save(f"./train_data/{data_mode}/split{i+1}/key_trn.npy",arr=key_trn)
                  np.save(f"./train_data/{data_mode}/split{i+1}/lbl_trn.npy",arr=lbl_trn)
                  np.save(f"./train_data/{data_mode}/split{i+1}/z_snd_trn.npy",arr=Z_snd_trn)
                  np.save(f"./train_data/{data_mode}/split{i+1}/z_img_trn.npy",arr=Z_img_trn)

              if path_val_dir.exists():
                np.save(f"./val_data/{data_mode}/split{i+1}/key_val.npy",arr=key_val)
                np.save(f"./val_data/{data_mode}/split{i+1}/lbl_val.npy",arr=lbl_val)
                np.save(f"./val_data/{data_mode}/split{i+1}/z_snd_val.npy",arr=Z_snd_val)
                np.save(f"./val_data/{data_mode}/split{i+1}/z_img_val.npy",arr=Z_img_val)
              else:
                  path_val_dir.mkdir(parents=True,exist_ok=True)
                  np.save(f"./val_data/{data_mode}/split{i+1}/key_val.npy",arr=key_val)
                  np.save(f"./val_data/{data_mode}/split{i+1}/lbl_val.npy",arr=lbl_val)
                  np.save(f"./val_data/{data_mode}/split{i+1}/z_snd_val.npy",arr=Z_snd_val)
                  np.save(f"./val_data/{data_mode}/split{i+1}/z_img_val.npy",arr=Z_img_val)

              train_data_at.add_dir(path_train_dir)
              val_data_at.add_dir(path_val_dir)

              run.log_artifact(val_data_at)
              print(f"----  Logged val data for {run.id}: {data_mode}/{i+1} ----")

              run.log_artifact(train_data_at)
              print(f"----  Logged train data for {run.id}: {data_mode}/{i+1} ----")


            Z_trn = dict(
                image=Z_img_trn,
                sound=Z_snd_trn,
                mean=(Z_img_trn + Z_snd_trn) / 2,
                concat=np.concatenate([Z_img_trn, Z_snd_trn], axis=1)
            )[data_mode]
            Z_val = dict(
                image=Z_img_val,
                sound=Z_snd_val,
                mean=(Z_img_val + Z_snd_val) / 2,
                concat=np.concatenate([Z_img_val, Z_snd_val], axis=1)
            )[data_mode]
            Z_test = dict(
                image=Z_img_test,
                sound=Z_snd_test,
                mean=(Z_img_test + Z_snd_test) / 2,
                concat=np.concatenate([Z_img_test, Z_snd_test], axis=1)
            )[data_mode]

            if 'Contrastive' in cfg.LossFunction:
                penalty = 'none'
            else:
                penalty = 'l2'
            lr = LogisticRegression(multi_class='multinomial', max_iter=500000, penalty=penalty, n_jobs=1)
            lr.fit(Z_trn, lbl_trn)
            prediction = lr.predict(Z_test)

            if Path("./lr_models").exists():
              with open(f'./lr_models/split{i+1}_{data_mode}_lr_model.pkl', 'wb') as file:
                pickle.dump(lr, file)
            else:
              Path("./lr_models").mkdir(parents=True)
              with open(f'./lr_models/split{i+1}_{data_mode}_lr_model.pkl', 'wb') as file:
                pickle.dump(lr, file)

            res = dict(data=data_mode)
            precision, recall, fscore, _ = precision_recall_fscore_support(lbl_test, prediction, average='weighted')
            res['precision'] = precision
            res['recall'] = recall
            res['fscore'] = fscore
            results.append(res)

     # log models
    models_artifact = wandb.Artifact("lr_models", type="model")
    models_artifact.add_dir("./lr_models",name="all_lr_models")
    run.log_artifact(models_artifact)
    print("---- saved all lr models ---")

    json_res = json.dumps(results)
    df = pd.read_json(json_res,orient="records")
    #pd.options.display.float_format = lambda x: f'{100*x:.2f}%'

    metrics = df.groupby(by="data").mean().loc["concat"]
    wandb.log({
        f'Advance/{metric.title()}': val for metric, val in metrics.items()
    })

    for data_mode, data in df.groupby(by='data').mean().T.items():
        metrics = data.to_dict()
        wandb.log({
            f'AdvanceDetailed/{data_mode}_{metric.title()}': val for metric, val in metrics.items()
        })

    res = df.melt(id_vars=['data']).groupby(['data', 'variable']).agg(['mean', 'std']).T


# 4 - Training

In [ ]:
dev = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda")

# parser = ArgumentParser(description='Test Advance Data')
# parser.add_argument("model", type=Path, help='folder containing the trained model')
# args = parser.parse_args()

#PATH_TO_MODELS = Path("./logs/run_vit_base_patch16_224_ft_in21k")

cfg.defrost()
cfg.merge_from_file(PATH_TO_MODELS / 'config.yml')
cfg.freeze()


#run_id = "test_advance_vit_base_32patch_224_ft_in21k_lr_v02_latest"
#assert run_id != ''

run = wandb.init(project='teste_audiovisual_advance', resume=True, id=run_id)

img_encoder   = get_model(cfg.ImageEncoder,
        batch_size=cfg.BatchSize, output_dim=cfg.LatentDim,
        model_id=cfg.ModelName
    )
snd_encoder   = get_model(cfg.ImageEncoder,
        batch_size=cfg.BatchSize, output_dim=cfg.LatentDim,
        model_id=cfg.ModelName
    )

loss_function = get_loss_function(cfg.LossFunction)(*cfg.LossArg)
model = FullModelWrapper(img_encoder, snd_encoder, loss_function)
model = model.to(dev)
model.eval()

model.load_state_dict(torch.load(PATH_TO_MODELS / 'checkpoints/best.pt', map_location=dev))
evaluate_advance(model,seed=seed,run=run,log_artifacts=True)
run.finish()

wandb: wandb version 0.16.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in /content/test_advance/wandb/run-20240127_040317-test_advance_vit_base_32patch_224_ft_in21k_lr_v02_best
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run test_advance_vit_base_32patch_224_ft_in21k_lr_v02_best
wandb: ⭐️ View project at https://wandb.ai/talissamoura/teste_audiovisual_advance
wandb: 🚀 View run at https://wandb.ai/talissamoura/teste_audiovisual_advance/runs/test_advance_vit_base_32patch_224_ft_in21k_lr_v02_best
  0%|          | 0/5075 [00:00<?, ?it/s]/usr/local/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this 

z_snd after stack: (5075, 32, 128)
z_snd after rearrange: (5075, 4096)
--- save lbl2idex file ---
--- logged labelidx file ---


wandb: Adding directory to artifact (./test_data)... Done. 0.1s


---- save test data for run test_advance_vit_base_32patch_224_ft_in21k_lr_v02_best ----
----  Logged test data for test_advance_vit_base_32patch_224_ft_in21k_lr_v02_best ----
Taking image as data basis.


  0%|          | 0/5 [00:00<?, ?it/s]

...

Done. 0.2s
wandb: Adding directory to artifact (./val_data/concat/split5)... Done. 0.0s


----  Logged val data for test_advance_vit_base_32patch_224_ft_in21k_lr_v02_best: concat/5 ----
----  Logged train data for test_advance_vit_base_32patch_224_ft_in21k_lr_v02_best: concat/5 ----


100%|██████████| 5/5 [1:04:46<00:00, 777.32s/it]
wandb: Adding directory to artifact (./lr_models)... Done. 0.1s
wandb: Waiting for W&B process to finish... (success).


---- saved all lr models ---


wandb: 
wandb: Run history:
wandb:                   Advance/Fscore ▁
wandb:                Advance/Precision ▁
wandb:                   Advance/Recall ▁
wandb:    AdvanceDetailed/concat_Fscore ▁
wandb: AdvanceDetailed/concat_Precision ▁
wandb:    AdvanceDetailed/concat_Recall ▁
wandb:     AdvanceDetailed/image_Fscore ▁
wandb:  AdvanceDetailed/image_Precision ▁
wandb:     AdvanceDetailed/image_Recall ▁
wandb:      AdvanceDetailed/mean_Fscore ▁
wandb:   AdvanceDetailed/mean_Precision ▁
wandb:      AdvanceDetailed/mean_Recall ▁
wandb:     AdvanceDetailed/sound_Fscore ▁
wandb:  AdvanceDetailed/sound_Precision ▁
wandb:     AdvanceDetailed/sound_Recall ▁
wandb: 
wandb: Run summary:
wandb:                   Advance/Fscore 0.85471
wandb:                Advance/Precision 0.85727
wandb:                   Advance/Recall 0.85616
wandb:    AdvanceDetailed/concat_Fscore 0.85471
wandb: AdvanceDetailed/concat_Precision 0.85727
wandb:    AdvanceDetailed/concat_Recall 0.85616
wandb:     AdvanceDetailed